In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# !wget http://www.gutenberg.org/cache/epub/11/pg11.txt

--2020-02-04 09:16:29--  http://www.gutenberg.org/cache/epub/11/pg11.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60422 (59K) [text/plain]
Saving to: ‘pg11.txt’

pg11.txt            100%[===================>]  59.01K  --.-KB/s    in 0.05s   

2020-02-04 09:16:29 (1.23 MB/s) - ‘pg11.txt’ saved [60422/60422]



In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:

filename = "/content/drive/My Drive/Datasets/wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [0]:
# create mapping of unique chars to integer
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:

n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  144507
Total Vocab:  45


In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  144407


In [0]:

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
# define the checkpoint
filepath="/content/drive/My Drive/Models/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
144407/144407 [==============================] - 165s 1ms/step - loss: 2.9499

Epoch 00001: loss improved from inf to 2.94989, saving model to /content/drive/My Drive/Models/weights-improvement-01-2.9499.hdf5
Epoch 2/20
144407/144407 [==============================] - 166s 1ms/step - loss: 2.7630

Epoch 00002: loss improved from 2.94989 to 2.76303, saving model to /content/drive/My Drive/Models/weights-improvement-02-2.7630.hdf5
Epoch 3/20
144407/144407 [==============================] - 165s 1ms/step - loss: 2.6604

Epoch 00003: loss improved from 2.76303 to 2.66040, saving model to /content/drive/My Drive/Models/weights-improvement-03-2.6604.hdf5
Epoch 4/20
144407/144407 [==============================] - 165s 1ms/step - loss: 2.5808

Epoch 00004: loss improved from 2.66040 to 2.58079, saving model to /content/drive/My Drive/Models/weights-improvement-04-2.5808.hdf5
Epoch 5/20
144407/144407 [==============================] - 163s 1ms/step - loss: 2.5160

Epoch 00005: loss 

In [0]:
# load the network weights
filename = "/content/drive/My Drive/Models/weights-improvement-20-1.9356.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([chr[value] for value in pattern]), "\"")

for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")







Seed:


TypeError: ignored